In [ ]:
!pip install umap-learn
!pip install adversarial-robustness-toolbox


In [4]:
import pandas as pd
import numpy as np
import umap
from umap import UMAP
import tensorflow as tf
import keras
from art.attacks.evasion import FastGradientMethod, CarliniLInfMethod
from tensorflow.keras.datasets import mnist, cifar10
from art.estimators.classification import KerasClassifier
from art.estimators.classification import TensorFlowV2Classifier
tf.compat.v1.disable_eager_execution()

In [1]:
# Load and preprocess data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0
y_test_cat = tf.keras.utils.to_categorical(y_test, num_classes=10)


In [ ]:
# Load model and create ART classifier
model = tf.keras.models.load_model('mnist_CNN_model.h5')
classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=10,
    input_shape=(28, 28, 1),
    loss_object=tf.keras.losses.CategoricalCrossentropy()
)

# Generate adversarial examples
epsilon = 0.1
fgsm = FastGradientMethod(classifier, eps=epsilon)
x_test_adv_fgsm = fgsm.generate(x_test)


In [ ]:
# Get predictions
preds_true = y_test  # True labels (already 0-9)
preds_original = np.argmax(model.predict(x_test), axis=1)
preds_adv = np.argmax(model.predict(x_test_adv_fgsm), axis=1)

# Create 5D UMAP embeddings
reducer = umap.UMAP(n_neighbors=15, n_components=2, min_dist=0.1, random_state=42)
UMAP_2d = reducer.fit_transform(x_test.reshape(len(x_test), -1))
adv_UMAP_2d = reducer.fit_transform(x_test_adv_fgsm.reshape(len(x_test_adv_fgsm), -1))

In [ ]:
# Create DataFrames for features
legitimate_df = pd.DataFrame(
    UMAP_2d,
    columns=[f"UMAP_{i}" for i in range(2)]
)

attack_df = pd.DataFrame(
    adv_UMAP_2d,
    columns=[f"Adv_UMAP_{i}" for i in range(2)]
)

# Create DataFrame for labels and results
results_df = pd.DataFrame({
    'Legit_Label': preds_true,
    'Predicted_Label': preds_original,
    'Attacked_Label': preds_adv,
    'Attack_Success': (preds_original != preds_adv).astype(int)
})

# Combine all DataFrames
output_df = pd.concat([legitimate_df, attack_df, results_df], axis=1)

# Ensure column order matches your example
column_order = [
    *[f"UMAP_{i}" for i in range(2)],
    'Legit_Label',
    *[f"Adv_UMAP_{i}" for i in range(2)],
    'Predicted_Label', 
    'Attacked_Label',
    'Attack_Success'
]
output_df = output_df[column_order]

# Save and display results
output_df.to_csv('adversarial_analysis_results.csv', index=False)
print(output_df.head())